In [264]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [265]:
class FoodDataset(Dataset):
    def __init__(self, filename, window=7, train=True, **kwargs):
        self.foods = {
            "chinese": ["black bean sauce noodle", "fried rice", "hotpot(huoguo)"], 
            "indian": ["curry rice"], 
            "western": ["pasta", "pizza", "fried chicken", "hamburger", "steak", "hotdog"], 
            "korean": ["korean bbq", "kimchi stew", "soybean paste stew","cold noodles", "bibimbap"],
            "mexican" : ["taco", "burrito"],
            "japanese": ["sushi", "donkatsu"],
            "vietnam": ["pho", "banh mi thit"]
        }
        if train:
            self.foods_cate_dict = {k: i for i, k in enumerate(sorted(list(self.foods.keys())))}
            all_foods = sorted([x for foods in self.foods.values() for x in foods])
            self.food_dict = {v: i for i, v in enumerate(all_foods)}
            self.workload_dict = {"No work": 0, "Low": 1, "Normal": 2, "High": 3}
        else:
            self.foods_cate_dict = kwargs["foods_cate_dict"]
            self.food_dict = kwargs["food_dict"]
            self.workload_dict = {"No work": 0, "Low": 1, "Normal": 2, "High": 3}
            
        with open(filename, "r") as file:
            data = file.readlines()[1:]
            
        numericalize = lambda x: (
            float(x[0]), int(x[1]), self.workload_dict.get(x[2]), 
            int(x[3]), self.foods_cate_dict.get(x[4]), self.food_dict.get(x[5])
        ) 
        # load data and numericalize
        # data: temperature, hometime, workload, earlymeeting, food
        data = torch.FloatTensor([numericalize(line.rstrip("\n").split(",")) for line in data])
        
        # input columns = 5
        y = []
        X = []
        for idx in torch.arange(1, len(data)+1-window):
            # add what category food i ate yesterday
            window_data = data[idx:idx+window, :]
            ate_yesterday = data[(idx-1):(idx-1+window), -2]
            X.append(torch.cat((window_data[:, :-2], ate_yesterday.view(-1, 1)), axis=1))
            y.append(window_data[-1, -1].item())
            
        self.y = torch.LongTensor(y)
        self.X = torch.stack(X)        
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [266]:
class FoodRecommendModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, bidirectional=True):
        super(FoodRecommendModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, 
                            num_layers=num_layers, batch_first=True, bidirectional=bidirectional)

        self.fc = nn.Linear(2*hidden_size, output_size)
        
    def forward(self, x):
        _, (hiddens, cells) = self.lstm(x)  # hiddens: n_direct, B, H > B, 
        last_t_hiddens = torch.cat([h for h in hiddens], dim=1)
        outputs = self.fc(last_t_hiddens)
        return outputs
    
    def predict(self, x):
        outputs = self.forward(x)
        return torch.argmax(outputs, axis=1)
    
    def recommand(self, x, recommand_num=3):
        outputs = self.forward(x)
        return outputs.sort(descending=True).view(-1)[:recommand_num]

In [267]:
def train_model(model, data_loader, loss_function, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    for inputs, targets in data_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        
        total_loss += loss.item()
        predicts = model.predict(inputs).cpu()
        total_correct += predicts.eq(targets.cpu()).sum().item()
        
        optimizer.step()
        
    train_loss = total_loss / len(data_loader)
    train_acc = total_correct / len(data_loader.dataset)
    return train_loss, train_acc

def test_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()
            predicts = model.predict(inputs).cpu()
            total_correct += predicts.eq(targets.cpu()).sum().item()
        
    test_loss = total_loss / len(data_loader)
    test_acc = total_correct / len(data_loader.dataset)
    model.to(device)
    return test_loss, test_acc

In [277]:
save_path_train="./dinner_records_train.csv"
save_path_test="./dinner_records_test.csv"
window = 7
batch_size = 32

In [278]:
food_train_dataset = FoodDataset(save_path_train, window)
preprocess_dict = {
    "foods_cate_dict": food_train_dataset.foods_cate_dict,
    "food_dict": food_train_dataset.food_dict,
}
food_test_dataset = FoodDataset(save_path_test, window, train=False, **preprocess_dict)

food_train_loader = DataLoader(food_train_dataset, batch_size=batch_size, shuffle=True)
food_test_loader = DataLoader(food_test_dataset, batch_size=batch_size, shuffle=True)

In [279]:
input_size = 5
hidden_size = 64
output_size = len(food_train_dataset.food_dict)
num_layers = 1
bidirectional = True
n_step = 500
wd_rate = 0.0001
device = "cuda" if torch.cuda.is_available() else "cpu"

In [284]:
model = FoodRecommendModel(input_size, hidden_size, output_size, num_layers, bidirectional).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=wd_rate)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=wd_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 250, 450], gamma=0.5)
loss_function = nn.CrossEntropyLoss()

In [285]:
best_acc = 0
for step in range(n_step):
    train_loss, train_acc = train_model(model, food_train_loader, loss_function, optimizer, device)
    test_loss, test_acc = test_model(model, food_test_loader, device)
    scheduler.step()
    if step % 50 == 0:
        print(f"[{step+1}] train - loss: {train_loss:.4f}, acc: {train_acc*100:.2f} %")
        print(f"[{step+1}]  test - loss: {test_loss:.4f}, acc: {test_acc*100:.2f} %")
    if test_acc >= best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_acc_model.pt")

[1] train - loss: 2.8438, acc: 11.48 %
[1]  test - loss: 2.8408, acc: 10.34 %
[51] train - loss: 1.8235, acc: 43.02 %
[51]  test - loss: 4.5844, acc: 8.28 %
[101] train - loss: 1.4034, acc: 54.77 %
[101]  test - loss: 5.3566, acc: 5.52 %
[151] train - loss: 1.2370, acc: 61.13 %
[151]  test - loss: 5.6360, acc: 8.97 %
[201] train - loss: 1.1358, acc: 62.79 %
[201]  test - loss: 6.2336, acc: 7.59 %
[251] train - loss: 1.0418, acc: 65.70 %
[251]  test - loss: 6.4667, acc: 6.90 %
[301] train - loss: 0.9176, acc: 70.82 %
[301]  test - loss: 6.6275, acc: 10.34 %
[351] train - loss: 0.8633, acc: 73.03 %
[351]  test - loss: 6.7720, acc: 8.97 %
[401] train - loss: 0.8282, acc: 74.69 %
[401]  test - loss: 6.5911, acc: 9.66 %
[451] train - loss: 0.7327, acc: 78.70 %
[451]  test - loss: 6.9624, acc: 8.97 %
